In [1]:
import pandas as pd
import numpy as np

In [2]:
ZAEBUC_AR_COR = pd.read_csv('ZAEBUC-v1.0/AR-all.extracted.corrected.analyzed.corrected-FINAL.tsv', encoding='utf_8',sep='\t')
words_df = ZAEBUC_AR_COR[ZAEBUC_AR_COR['Word'].notna()]

In [113]:
ZAEBUC_AR_ALL = pd.read_csv('ZAEBUC-v1.0/AR-all.alignment-FINAL.tsv', encoding='utf_8',sep='\t')

In [114]:
ZAEBUC_AR_ALL


,Document,Raw,Corrected,Operation
0,AR-030-268469,وسائل,وسائل,NO_CHANGE
1,AR-030-268469,التواصل,التواصل,NO_CHANGE
2,AR-030-268469,الاجتماعي,الاجتماعي,NO_CHANGE
3,AR-030-268469,لها,لها,NO_CHANGE
4,AR-030-268469,اضرار,أضرار,EDIT
...,...,...,...,...
33761,AR-130-99787,المجتمع,المجتمع,NO_CHANGE
33762,AR-130-99787,و,NaN,DELETE
33763,AR-130-99787,كيفية,وكيفية,EDIT
33764,AR-130-99787,إستعمالهم,استعمالهم,EDIT


In [115]:
ZAEBUC_AR_ALL['Operation'].unique()

array(['NO_CHANGE', 'EDIT', 'DELETE', 'INSERT'], dtype=object)

In [144]:
error_rate_df = ZAEBUC_AR_ALL[['Document','Operation']].groupby('Document').aggregate({'Operation': (lambda x: 1- np.sum(x=='NO_CHANGE')/len(x) )}).rename(columns = {'Operation':'error_rate'})

In [145]:
error_rate_df

,error_rate
Document,
AR-030-268469,0.455621
AR-030-386369,0.237179
AR-030-81027,0.585366
AR-030-81757,0.493927
AR-030-83625,0.428571
...,...
AR-130-99351,0.245283
AR-130-99438,0.059322
AR-130-99442,0.311475


In [5]:
tokenized_essays_df = words_df[['Document','Auto_Tokenization']].groupby(by = 'Document').agg({'Auto_Tokenization': ' '.join})
tokenized_essays_df['Auto_Tokenization'] = tokenized_essays_df['Auto_Tokenization'].apply(lambda x : x.replace('+', ' '))

In [6]:
tokenized_essays_df

,Auto_Tokenization
Document,
AR-030-268469,وسائل التواصل الاجتماعي ل ها أضرار و فوائد كثي...
AR-030-386369,تعد وسائل التواصل الاجتماعي من أكبر المؤثرات ع...
AR-030-81027,قام انتشار وسائل التواصل الاجتماعي ب شكل كبير ...
AR-030-81757,وسائل التواصل الاجتماعي لقد تطورت وسائل المعرف...
AR-030-83625,من أشهر وسائل الاتصال ب الآخرين هي الاجتماعية .
...,...
AR-130-99351,ظهور الأجهزة الإلكترونية أدى إلى ظهور وسائل ال...
AR-130-99438,وسائل التواصل الاجتماعي منذ انتشار وسائل التوا...
AR-130-99442,وسائل التواصل الاجتماعي إن التواصل الاجتماعي ل...


In [98]:
# def tokenize(doc):
#     doc = doc.split(' ')
#     doc = tokenizer.tokenize(doc)
#     tokens = []
#     for word in doc:
#         word = word.replace('+_', ',').replace('_+',',').split(',')
#         for tok in word:
#             tokens.append(tok)
#     return tokens

In [7]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Applying TFIDF
vectorizer = TfidfVectorizer(ngram_range = (1, 1) )
doc2vec = vectorizer.fit_transform(tokenized_essays_df['Auto_Tokenization'])
doc2vec = (doc2vec.toarray())
print("\n\nScores : \n", doc2vec)



Scores : 
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [8]:
tokenized_essays_df['Auto_POS'] = words_df[['Document', 'Auto_POS']].groupby(by = 'Document', as_index = True).agg({'Auto_POS': ' '.join})
tokenized_essays_df['Auto_POS'] = tokenized_essays_df['Auto_POS'].apply(lambda x: x.replace('+', ' '))

In [11]:
# Applying TFIDF
vectorizer = TfidfVectorizer(ngram_range = (1, 2))
pos2vec = vectorizer.fit_transform(tokenized_essays_df['Auto_POS'])
pos2vec = (pos2vec.toarray())
print("\n\nScores : \n", pos2vec)



Scores : 
 [[0.19436809 0.         0.07920732 ... 0.         0.         0.02296755]
 [0.29714894 0.0171133  0.08351155 ... 0.         0.         0.        ]
 [0.32720461 0.         0.08333737 ... 0.         0.         0.        ]
 ...
 [0.1976791  0.04306373 0.07004921 ... 0.         0.0155246  0.        ]
 [0.2220965  0.         0.0808098  ... 0.         0.02865505 0.        ]
 [0.26979631 0.02503347 0.07635085 ... 0.         0.         0.        ]]


In [9]:
import xmltodict
docs = ZAEBUC_AR_COR['Document'].apply(lambda x: x if x.startswith('<') else np.nan).dropna()

grades = []
word_count = []

for xml in docs:
    if xml != "</doc>":
        doc = xmltodict.parse(xml)
        grades.append(doc["doc"]["@CEFR"])
        word_count.append(doc["doc"]["@word_count"])

In [147]:
X = np.concatenate((doc2vec,pos2vec, np.array(word_count).reshape(-1,1), np.array(error_rate_df['error_rate']).reshape(-1,1) ), axis = 1).astype(float)

In [148]:
# SVM classifier for X and grades

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix

X_train, X_test, y_train, y_test = train_test_split(X, grades, test_size = 0.20)

svclassifier = SVC(kernel='linear')
svclassifier.fit(X_train, y_train)

y_pred = svclassifier.predict(X_test)

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))



[[ 2  0  1  0  0]
 [ 0  0  2  0  0]
 [ 2  0 20  1  0]
 [ 0  0  3 10  0]
 [ 0  0  0  2  0]]
              precision    recall  f1-score   support

          A0       0.50      0.67      0.57         3
          A2       0.00      0.00      0.00         2
          B1       0.77      0.87      0.82        23
          B2       0.77      0.77      0.77        13
          C1       0.00      0.00      0.00         2

    accuracy                           0.74        43
   macro avg       0.41      0.46      0.43        43
weighted avg       0.68      0.74      0.71        43



/Users/sanad/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sanad/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sanad/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [14]:
grades = list(map(lambda x:'A0' if (x == 'Unassessable') else x, grades))


In [15]:
def grades_to_num(grades):
    num_grades = []
    for x in grades:
        match x:
            case 'A0':
                num_grades.append(0)
            case 'A1':
                num_grades.append(1)
            case 'A2':
                num_grades.append(2)
            case 'B1':
                num_grades.append(3)
            case 'B2':
                num_grades.append(4)
            case 'C1':
                num_grades.append(5)
            case 'C2':
                num_grades.append(6)
    return num_grades
num_grades = grades_to_num(grades)

In [149]:
X_train, X_test, y_train, y_test = train_test_split(X, num_grades, test_size = 0.20, stratify = num_grades)

In [101]:
## given a dataset X and grades y, return a dataset of pair-wise differences and labels (+,-) 
def to_pairs(X, y):
    paired_X = list()
    paired_y = list()
    for i in range(len(X)):
        for k in range(i+1, len(X), 1):
                paired_X.append(np.subtract(X[i], X[k]))
                paired_y.append(y[i] > y[k])
    return paired_X, paired_y

In [150]:
X_train_diff, y_train_diff = to_pairs(X_train, y_train)

In [151]:
X_test_diff, y_test_diff = to_pairs(X_test, y_test)

In [152]:
from sklearn.model_selection import GridSearchCV
# param_grid = {'C': [0.1,1, 10, 100], 'kernel' : ['linear']}

# grid = GridSearchCV(SVC(),param_grid,refit=True,verbose=2)
# grid.fit(X_train_diff,y_train_diff)

svclassifier = SVC(kernel='linear')
svclassifier.fit(X_train_diff, y_train_diff)

y_pred = svclassifier.predict(X_test_diff)

print(confusion_matrix(y_test_diff,y_pred))
print(classification_report(y_test_diff,y_pred))



[[622  60]
 [111 110]]
              precision    recall  f1-score   support

       False       0.85      0.91      0.88       682
        True       0.65      0.50      0.56       221

    accuracy                           0.81       903
   macro avg       0.75      0.70      0.72       903
weighted avg       0.80      0.81      0.80       903



In [153]:
from sklearn.linear_model import LinearRegression

svc_fitted_X_train = svclassifier.coef_ @ np.transpose(X_train)
svc_fitted_X_train = svc_fitted_X_train.reshape(-1,1) 
svc_fitted_X_test = svclassifier.coef_ @ np.transpose(X_test)
svc_fitted_X_test = svc_fitted_X_test.reshape(-1,1) 

lm= LinearRegression(fit_intercept=True).fit(svc_fitted_X_train, y_train)
print(lm.predict(svc_fitted_X_train))
print(y_train)

[0.70136116 4.04085761 2.92069347 4.04101689 4.047345   4.04357734
 2.91613167 5.16787871 2.92502935 4.04269002 4.04229931 2.92523018
 2.92176802 4.04505568 2.92519778 4.0493465  2.92262226 2.92080762
 2.92018631 4.04244097 5.16611018 2.92251306 2.92056449 4.04514666
 4.05080357 2.92312745 2.92596179 2.92659206 2.92062573 2.92068064
 2.92545877 1.8053114  2.92194922 4.04410498 1.80724442 2.92192447
 2.91991227 4.04826155 4.04857454 2.92258587 2.92086489 2.92348876
 4.04406268 4.04659006 2.92395006 4.04452296 0.68972742 2.92366657
 2.91505458 5.16360668 2.91937368 2.917363   2.91674852 4.04750993
 2.92596917 4.04345289 5.1655644  4.04432169 4.04997626 2.92143784
 2.91621682 5.16627521 2.91711468 4.05041687 2.92418323 2.92325586
 2.92690098 4.04737717 2.92662435 1.80157652 2.9229217  4.04799789
 2.92329069 2.92362684 2.92450547 2.92232631 2.92304793 2.92563445
 5.17288191 2.91569915 2.92182398 2.91742176 4.04590192 2.9187649
 5.1667127  2.9197145  4.04134472 4.0469807  2.92461923 4.04694

In [154]:
print(np.concatenate((np.array(lm.predict(svc_fitted_X_test)).reshape(-1,1), np.array(y_test).reshape(-1,1)), axis = 1))

[[3.11403801 3.        ]
 [3.66988224 3.        ]
 [3.19526027 3.        ]
 [3.68525779 3.        ]
 [3.97268812 4.        ]
 [2.99957639 3.        ]
 [3.58183605 3.        ]
 [3.54325607 4.        ]
 [1.6514533  2.        ]
 [3.84400754 4.        ]
 [3.46003881 3.        ]
 [3.20890867 4.        ]
 [3.06546325 3.        ]
 [3.58727746 4.        ]
 [3.04185278 3.        ]
 [3.00173177 3.        ]
 [3.90402768 4.        ]
 [3.02947964 0.        ]
 [3.36087306 3.        ]
 [2.76261129 2.        ]
 [4.06777204 4.        ]
 [3.57963062 4.        ]
 [3.51854575 3.        ]
 [4.31810252 4.        ]
 [2.68096896 3.        ]
 [3.75726252 4.        ]
 [3.05665264 4.        ]
 [2.78215381 3.        ]
 [3.2666612  3.        ]
 [3.72336252 4.        ]
 [3.45763205 4.        ]
 [3.27444785 4.        ]
 [3.08822743 3.        ]
 [3.15551354 3.        ]
 [4.09474819 5.        ]
 [2.48587472 3.        ]
 [3.55044447 3.        ]
 [4.47608643 5.        ]
 [3.70220535 4.        ]
 [2.81434992 3.        ]


In [155]:
predicted_grades = np.floor(np.array(lm.predict(svc_fitted_X_test)).reshape(-1,1) + 0.5)
print(np.concatenate((predicted_grades, np.array(y_test).reshape(-1,1)), axis = 1))

[[3. 3.]
 [4. 3.]
 [3. 3.]
 [4. 3.]
 [4. 4.]
 [3. 3.]
 [4. 3.]
 [4. 4.]
 [2. 2.]
 [4. 4.]
 [3. 3.]
 [3. 4.]
 [3. 3.]
 [4. 4.]
 [3. 3.]
 [3. 3.]
 [4. 4.]
 [3. 0.]
 [3. 3.]
 [3. 2.]
 [4. 4.]
 [4. 4.]
 [4. 3.]
 [4. 4.]
 [3. 3.]
 [4. 4.]
 [3. 4.]
 [3. 3.]
 [3. 3.]
 [4. 4.]
 [3. 4.]
 [3. 4.]
 [3. 3.]
 [3. 3.]
 [4. 5.]
 [2. 3.]
 [4. 3.]
 [4. 5.]
 [4. 4.]
 [3. 3.]
 [4. 4.]
 [3. 3.]
 [3. 3.]]


In [156]:
np.corrcoef(np.floor(np.array(lm.predict(svc_fitted_X_test)) + 0.5), y_test)

array([[1.        , 0.53693014],
       [0.53693014, 1.        ]])

In [157]:
cm = confusion_matrix(y_test, np.floor(np.array(lm.predict(svc_fitted_X_test)) + 0.5))
print(cm)

[[ 0  0  1  0  0]
 [ 0  1  1  0  0]
 [ 0  1 16  5  0]
 [ 0  0  4 12  0]
 [ 0  0  0  2  0]]


In [158]:
print(classification_report(y_test,np.floor(np.array(lm.predict(svc_fitted_X_test)) + 0.5)))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           2       0.50      0.50      0.50         2
           3       0.73      0.73      0.73        22
           4       0.63      0.75      0.69        16
           5       0.00      0.00      0.00         2

    accuracy                           0.67        43
   macro avg       0.37      0.40      0.38        43
weighted avg       0.63      0.67      0.65        43



/Users/sanad/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sanad/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sanad/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Linear Classifier (SVC)

In [169]:
svm_model_linear = SVC(kernel = 'linear', C = 1).fit(svc_fitted_X_train, y_train) 
svm_predictions = svm_model_linear.predict(svc_fitted_X_test)

# model accuracy for X_test   
accuracy = svm_model_linear.score(svc_fitted_X_test, y_test) 
print('accuracy= {}'.format(accuracy))
# creating a confusion matrix 
cm = confusion_matrix(y_test, svm_predictions)
print(cm)

accuracy= 0.6976744186046512
[[ 0  0  1  0  0]
 [ 0  1  1  0  0]
 [ 0  0 17  5  0]
 [ 0  0  4 12  0]
 [ 0  0  0  2  0]]


In [170]:
print(classification_report(y_test,svm_predictions))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           2       1.00      0.50      0.67         2
           3       0.74      0.77      0.76        22
           4       0.63      0.75      0.69        16
           5       0.00      0.00      0.00         2

    accuracy                           0.70        43
   macro avg       0.47      0.40      0.42        43
weighted avg       0.66      0.70      0.67        43



/Users/sanad/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sanad/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sanad/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
